# Recommender systems - Neural Collaborative Filtering
> Demo

- toc: true 
- badges: true
- comments: true
- hide: true
- categories: [demo, neural networks, deep learning, recommender systems, paper]

Download dependencies and run `tensorboard` in the background:


In [1]:
"""
!pip install tensorflow lightfm pandas

%load_ext tensorboard
!tensorboard --logdir 2020-09-11-neural_collaborative_filter/logs &
"""

'\n!pip install tensorflow lightfm pandas\n\n%load_ext tensorboard\n!tensorboard --logdir 2020-09-11-neural_collaborative_filter/logs &\n'

In [2]:
#hide
import datetime
import os

import lightfm
import numpy as np
import pandas as pd
import tensorflow as tf
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from scipy import sparse
from tensorboard import notebook

In [3]:
#hide
print(f"Tensorflow version: {tf.__version__}")
print(f"LightFM version: {lightfm.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

Tensorflow version: 2.5.0-dev20210122
LightFM version: 1.16
Pandas version: 1.1.4
Numpy version: 1.19.2


In [4]:
TOP_K = 5
N_EPCOCHS = 10

# Data

![](https://raw.githubusercontent.com/murilo-cunha/inteligencia-superficial/master/images/2020-09-11-neural_collaborative_filter/matrix_factorization_with_alpha.png "Credit: https://developers.google.com/machine-learning/recommendation/collaborative/basics")

In [5]:
#hide_input
data = fetch_movielens(min_rating=3.0)

print("Interaction matrix:")
print(data["train"].toarray()[:10, :10])

Interaction matrix:
[[5 3 4 3 3 5 4 0 5 3]
 [4 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [4 0 0 0 0 0 0 4 4 0]
 [0 0 0 5 0 0 5 5 5 4]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 4 0 0 0]
 [4 0 0 4 0 0 0 0 4 0]]


In [6]:
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 74627 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 7893 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}

In [7]:
#collapse
for dataset in ["test", "train"]:
    data[dataset] = (data[dataset].toarray() > 0).astype("int8")  # astype("int8")을 통해 조건값 이진화.
    
# Make the ratings binary
print("Interaction matrix:")
print(data["train"][:10, :10])

print("\nRatings:")
unique_ratings = np.unique(data["train"])
print(unique_ratings)

Interaction matrix:
[[1 1 1 1 1 1 1 0 1 1]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 1 0]
 [0 0 0 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 1 0 0 0 0 1 0]]

Ratings:
[0 1]


In [8]:
from typing import List

def wide_to_long(wide: np.array, possible_ratings: List[int]) -> np.array:
    """Go from wide table to long.
    :param wide: wide array with user-item interactions
    :param possible_ratings: list of possible ratings that we may have."""
    
    def _get_ratings(arr: np.array, rating: int) -> np.array:
        """Generate long array for the rating provided
        :param arr: wide array with user-item interactions
        :param rating: the rating that we are interested"""
        idx = np.where(arr == rating)
        return np.vstack(
            (idx[0], idx[1], np.ones(idx[0].size, dtype="int8") * rating)  # 
        ).T  # .T: 행과열 전치
    
    long_arrays = []
    for r in possible_ratings:
        long_arrays.append(_get_ratings(wide, r))
        
    return np.vstack(long_arrays)

In [9]:
long_train = wide_to_long(data["train"], unique_ratings)
df_train = pd.DataFrame(long_train, columns=['user_id', 'item_id', 'interaction'])

In [10]:
long_train

array([[   0,    7,    0],
       [   0,   10,    0],
       [   0,   19,    0],
       ...,
       [ 942, 1187,    1],
       [ 942, 1227,    1],
       [ 942, 1329,    1]])

In [11]:
#hide_input
print("All interactions:")
df_train

All interactions:


,user_id,item_id,interaction
0,0,7,0
1,0,10,0
2,0,19,0
3,0,20,0
4,0,26,0
...,...,...,...
1586121,942,1043,1
1586122,942,1073,1
1586123,942,1187,1
1586124,942,1227,1


In [12]:
#hide_input
print("Only positive interactions:")
df_train[df_train["interaction"] > 0].head()

Only positive interactions:


,user_id,item_id,interaction
1511499,0,0,1
1511500,0,1,1
1511501,0,2,1
1511502,0,3,1
1511503,0,4,1


# The model (Neural Collaborative Filtering)

<center><img src="https://raw.githubusercontent.com/murilo-cunha/inteligencia-superficial/master/images/2020-09-11-neural_collaborative_filter/ncf_all_with_alpha.png" width="70%" url="https://developers.google.com/machine-learning/recommendation/collaborative/basics" description="Fonte: https://developers.google.com/machine-learning/recommendation/collaborative/basics" /> </center>

In [ ]:
import 